In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku 

In [13]:
# Create tokenizer
path_sonnets = "data/sonnet/sonnets.txt"
data = open(path_sonnets).read()
lines = data.lower().split("\n")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
total_words = len(tokenizer.word_index) + 1

In [26]:
# Divide sentences to list of n-gram
input_sequences = []
for line in lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(tokenizern_list[:i+1])

In [27]:
import numpy as np

# Padding sentences with type pre
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [28]:
# Get the last element as label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

In [38]:
model = Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(10))
model.add(Dense(64, kernel_regularizer='l1'))
model.add(Dense(total_words,activation="softmax"))
# Pick an optimizer
model.compile(loss="binary_crossentropy", optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 100)            321100    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 5, 300)            301200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 300)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10)                12440     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                704       
_________________________________________________________________
dense_3 (Dense)              (None, 3211)              208715    
Total params: 844,159
Trainable params: 844,159
Non-trainable params: 0
________________________________________________

In [40]:
history = model.fit(predictors, label, epochs=5, batch_size=128, verbose=1)

Epoch 1/5
121/121 [==============================] - 8s 63ms/step - loss: 0.0014 - accuracy: 0.4425
Epoch 2/5
121/121 [==============================] - 8s 64ms/step - loss: 0.0014 - accuracy: 0.4425
Epoch 3/5
121/121 [==============================] - 8s 67ms/step - loss: 0.0014 - accuracy: 0.4425
Epoch 4/5
121/121 [==============================] - 8s 67ms/step - loss: 0.0014 - accuracy: 0.4425
Epoch 5/5
121/121 [==============================] - 8s 64ms/step - loss: 0.0014 - accuracy: 0.4425


In [36]:
seed_text = "Help me Obi Wan Kenobi"
next_words = 5
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

Help me Obi Wan Kenobi increase increase increase increase increase
